<a href="https://colab.research.google.com/github/nduas77/AndriodQuizApplication/blob/master/nduas77%20resources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import json
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict

data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive

<module 'google.colab.drive' from '/usr/local/lib/python3.6/dist-packages/google/colab/drive.py'>

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1u2ZaH9py7IrjjcLjCrUv4RE0XdsRR93r
Downloaded content "Sample upload file content"


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

In [0]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 14mFVjHQ2vcbA4QgoQItsg1JdbPxGBZI8


In [0]:
# Download the file we just uploaded.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'target_file_id'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
sh = gc.create('A new spreadsheet')

In [0]:
# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1YUb1ASY9ZMVtTgQh2z5Acycrxp3At7CMNyhXjJwcYfU',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

In [0]:
# Open our new sheet and read some data.
worksheet = gc.open('A new spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

[['9', '2', '3'], ['5', '10', '8']]


,0,1,2
0,9,2,3
1,5,10,8


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

In [0]:
# First, we need to set our project. Replace the assignment below
# with your project ID.
project_id = 'nduas77'

In [0]:
!gcloud config set project {project_id}

Updated property [core/project].


In [0]:
import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-2ea7ab4c-09ca-11e9-b276-0242ac1c0002/...


In [0]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
-
Operation completed over 1 objects/14.0 B.                                       


In [0]:
# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [0]:
# Download the file.
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket-2ea7ab4c-09ca-11e9-b276-0242ac1c0002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

In [0]:
# Download the file.
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket-2ea7ab4c-09ca-11e9-b276-0242ac1c0002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

In [0]:
# Download the file.
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket-2ea7ab4c-09ca-11e9-b276-0242ac1c0002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

In [0]:
# Inspect the file we downloaded to /tmp
!cat /tmp/downloaded_from_gcs.txt

cat: /tmp/downloaded_from_gcs.txt: No such file or directory


In [0]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

name 'take_photo' is not defined


In [0]:
%%javascript
const listenerChannel = new BroadcastChannel('channel');
listenerChannel.onmessage = (msg) => {
  const div = document.createElement('div');
  div.textContent = msg.data;
  document.body.appendChild(div);
};

In [0]:
%%javascript
const senderChannel = new BroadcastChannel('channel');
senderChannel.postMessage('Hello world!');

<IPython.core.display.Javascript object>

In [0]:
import IPython
js_code = '''
document.querySelector("#output-area").appendChild(document.createTextNode("hello world!"));
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [0]:
#TO DOWNLOAD TO LACAL FOLDERS
from google.colab import files

with open('TEST.txt', 'w') as f:
  f.write('some content')

files.download('TEST.txt')

In [0]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  window.someValue = new Promise(resolve => {
    setTimeout(() => {
      resolve("hello world!");
    }, 100);
  });
'''))


value = output.eval_js('someValue');
value

<IPython.core.display.Javascript object>

'hello world!'

In [0]:
#@title Give me a name {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


In [0]:
!pip install -q matplotlib-venn

In [0]:
!apt-get -qq install -y libfluidsynth1

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

#worksheet = gc.open('Your spreadsheet name').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

[['9', '2', '3'], ['5', '10', '8']]


,0,1,2
0,9,2,3
1,5,10,8


In [0]:
!apt-get -qq install python-cartopy python3-cartopy
import cartopy

Selecting previously unselected package python-pkg-resources.
(Reading database ... 110847 files and directories currently installed.)
Preparing to unpack .../0-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-pyshp.
Preparing to unpack .../1-python-pyshp_1.2.12+ds-1_all.deb ...
Unpacking python-pyshp (1.2.12+ds-1) ...
Selecting previously unselected package python-shapely.
Preparing to unpack .../2-python-shapely_1.6.4-1_amd64.deb ...
Unpacking python-shapely (1.6.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-cartopy:amd64.
Preparing to unpack .../4-python-cartopy_0.14.2+dfsg1-2build3_amd64.deb ...
Unpacking python-cartopy:amd64 (0.14.2+dfsg1-2build3) ...
Selecting previously unselected package python3-pkg-resources.
Preparing to unpack .../5-pyth

In [66]:
from vega_datasets import data
stocks = data.stocks()

import altair as alt
alt.Chart(stocks).mark_line().encode(
  x='date:T',
  y='price',
  color='symbol'
).interactive(bind_y=False)

Chart({
  data:     symbol       date   price
  0     MSFT 2000-01-01   39.81
  1     MSFT 2000-02-01   36.35
  2     MSFT 2000-03-01   43.22
  3     MSFT 2000-04-01   28.37
  4     MSFT 2000-05-01   25.45
  5     MSFT 2000-06-01   32.54
  6     MSFT 2000-07-01   28.40
  7     MSFT 2000-08-01   28.40
  8     MSFT 2000-09-01   24.53
  9     MSFT 2000-10-01   28.02
  10    MSFT 2000-11-01   23.34
  11    MSFT 2000-12-01   17.65
  12    MSFT 2001-01-01   24.84
  13    MSFT 2001-02-01   24.00
  14    MSFT 2001-03-01   22.25
  15    MSFT 2001-04-01   27.56
  16    MSFT 2001-05-01   28.14
  17    MSFT 2001-06-01   29.70
  18    MSFT 2001-07-01   26.93
  19    MSFT 2001-08-01   23.21
  20    MSFT 2001-09-01   20.82
  21    MSFT 2001-10-01   23.65
  22    MSFT 2001-11-01   26.12
  23    MSFT 2001-12-01   26.95
  24    MSFT 2002-01-01   25.92
  25    MSFT 2002-02-01   23.73
  26    MSFT 2002-03-01   24.53
  27    MSFT 2002-04-01   21.26
  28    MSFT 2002-05-01   20.71
  29    MSFT 2002-06-01   22.25
  ..     ...        ...     ...
  530   AAPL 2007-10-01  189.95
  531   AAPL 2007-11-01  182.22
  532   AAPL 2007-12-01  198.08
  533   AAPL 2008-01-01  135.36
  534   AAPL 2008-02-01  125.02
  535   AAPL 2008-03-01  143.50
  536   AAPL 2008-04-01  173.95
  537   AAPL 2008-05-01  188.75
  538   AAPL 2008-06-01  167.44
  539   AAPL 2008-07-01  158.95
  540   AAPL 2008-08-01  169.53
  541   AAPL 2008-09-01  113.66
  542   AAPL 2008-10-01  107.59
  543   AAPL 2008-11-01   92.67
  544   AAPL 2008-12-01   85.35
  545   AAPL 2009-01-01   90.13
  546   AAPL 2009-02-01   89.31
  547   AAPL 2009-03-01  105.12
  548   AAPL 2009-04-01  125.83
  549   AAPL 2009-05-01  135.81
  550   AAPL 2009-06-01  142.43
  551   AAPL 2009-07-01  163.39
  552   AAPL 2009-08-01  168.21
  553   AAPL 2009-09-01  185.35
  554   AAPL 2009-10-01  188.50
  555   AAPL 2009-11-01  199.91
  556   AAPL 2009-12-01  210.73
  557   AAPL 2010-01-01  192.06
  558   AAPL 2010-02-01  204.62
  559   AAPL 2010-03-01  223.02
  
  [560 rows x 3 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'symbol'
    }),
    x: X({
      shorthand: 'date:T'
    }),
    y: Y({
      shorthand: 'price'
    })
  }),
  mark: 'line',
  selection: SelectionMapping({
    selector001: SelectionDef({
      bind: 'scales',
      encodings: ['x'],
      type: 'interval'
    })
  })
})

In [67]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_bar().encode(
  x=alt.X('Miles_per_Gallon', bin=True),
  y='count()',
  color='Origin'
)

Chart({
  data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
  0            12.0          8         307.0       130.0              18.0   
  1            11.5          8         350.0       165.0              15.0   
  2            11.0          8         318.0       150.0              18.0   
  3            12.0          8         304.0       150.0              16.0   
  4            10.5          8         302.0       140.0              17.0   
  5            10.0          8         429.0       198.0              15.0   
  6             9.0          8         454.0       220.0              14.0   
  7             8.5          8         440.0       215.0              14.0   
  8            10.0          8         455.0       225.0              14.0   
  9             8.5          8         390.0       190.0              15.0   
  10           17.5          4         133.0       115.0               NaN   
  11           11.5          8         350.0       165.0               NaN   
  12           11.0          8         351.0       153.0               NaN   
  13           10.5          8         383.0       175.0               NaN   
  14           11.0          8         360.0       175.0               NaN   
  15           10.0          8         383.0       170.0              15.0   
  16            8.0          8         340.0       160.0              14.0   
  17            8.0          8         302.0       140.0               NaN   
  18            9.5          8         400.0       150.0              15.0   
  19           10.0          8         455.0       225.0              14.0   
  20           15.0          4         113.0        95.0              24.0   
  21           15.5          6         198.0        95.0              22.0   
  22           15.5          6         199.0        97.0              18.0   
  23           16.0          6         200.0        85.0              21.0   
  24           14.5          4          97.0        88.0              27.0   
  25           20.5          4          97.0        46.0              26.0   
  26           17.5          4         110.0        87.0              25.0   
  27           14.5          4         107.0        90.0              24.0   
  28           17.5          4         104.0        95.0              25.0   
  29           12.5          4         121.0       113.0              26.0   
  ..            ...        ...           ...         ...               ...   
  376          18.6          4         112.0        88.0              27.0   
  377          18.0          4         112.0        88.0              34.0   
  378          16.2          4         112.0        85.0              31.0   
  379          16.0          4         135.0        84.0              29.0   
  380          18.0          4         151.0        90.0              27.0   
  381          16.4          4         140.0        92.0              24.0   
  382          20.5          4         151.0         NaN              23.0   
  383          15.3          4         105.0        74.0              36.0   
  384          18.2          4          91.0        68.0              37.0   
  385          17.6          4          91.0        68.0              31.0   
  386          14.7          4         105.0        63.0              38.0   
  387          17.3          4          98.0        70.0              36.0   
  388          14.5          4         120.0        88.0              36.0   
  389          14.5          4         107.0        75.0              36.0   
  390          16.9          4         108.0        70.0              34.0   
  391          15.0          4          91.0        67.0              38.0   
  392          15.7          4          91.0        67.0              32.0   
  393          16.2          4          91.0        67.0              38.0   
  394          16.4          6         181.0       110.0              25.0   
  395   

In [68]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

points = alt.Chart(cars).mark_point().encode(
  x='Year:T',
  y='Miles_per_Gallon',
  color='Origin'
).properties(
  width=800
)

lines = alt.Chart(cars).mark_line().encode(
  x='Year:T',
  y='mean(Miles_per_Gallon)',
  color='Origin'
).properties(
  width=800
).interactive(bind_y=False)
              
points + lines

LayerChart({
  layer: [Chart({
    data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
    0            12.0          8         307.0       130.0              18.0   
    1            11.5          8         350.0       165.0              15.0   
    2            11.0          8         318.0       150.0              18.0   
    3            12.0          8         304.0       150.0              16.0   
    4            10.5          8         302.0       140.0              17.0   
    5            10.0          8         429.0       198.0              15.0   
    6             9.0          8         454.0       220.0              14.0   
    7             8.5          8         440.0       215.0              14.0   
    8            10.0          8         455.0       225.0              14.0   
    9             8.5          8         390.0       190.0              15.0   
    10           17.5          4         133.0       115.0               NaN   
    11           11.5          8         350.0       165.0               NaN   
    12           11.0          8         351.0       153.0               NaN   
    13           10.5          8         383.0       175.0               NaN   
    14           11.0          8         360.0       175.0               NaN   
    15           10.0          8         383.0       170.0              15.0   
    16            8.0          8         340.0       160.0              14.0   
    17            8.0          8         302.0       140.0               NaN   
    18            9.5          8         400.0       150.0              15.0   
    19           10.0          8         455.0       225.0              14.0   
    20           15.0          4         113.0        95.0              24.0   
    21           15.5          6         198.0        95.0              22.0   
    22           15.5          6         199.0        97.0              18.0   
    23           16.0          6         200.0        85.0              21.0   
    24           14.5          4          97.0        88.0              27.0   
    25           20.5          4          97.0        46.0              26.0   
    26           17.5          4         110.0        87.0              25.0   
    27           14.5          4         107.0        90.0              24.0   
    28           17.5          4         104.0        95.0              25.0   
    29           12.5          4         121.0       113.0              26.0   
    ..            ...        ...           ...         ...               ...   
    376          18.6          4         112.0        88.0              27.0   
    377          18.0          4         112.0        88.0              34.0   
    378          16.2          4         112.0        85.0              31.0   
    379          16.0          4         135.0        84.0              29.0   
    380          18.0          4         151.0        90.0              27.0   
    381          16.4          4         140.0        92.0              24.0   
    382          20.5          4         151.0         NaN              23.0   
    383          15.3          4         105.0        74.0              36.0   
    384          18.2          4          91.0        68.0              37.0   
    385          17.6          4          91.0        68.0              31.0   
    386          14.7          4         105.0        63.0              38.0   
    387          17.3          4          98.0        70.0              36.0   
    388          14.5          4         120.0        88.0              36.0   
    389          14.5          4         107.0        75.0              36.0   
    390          16.9          4         108.0        70.0              34.0   
    391          15.0          4          91.0        67.0              38.0   
    392          15.7          4          91.0        67.0              32.0   
    393          16.2          4          9

In [69]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

interval = alt.selection_interval()

points = alt.Chart(cars).mark_point().encode(
  x='Horsepower',
  y='Miles_per_Gallon',
  color=alt.condition(interval, 'Origin', alt.value('lightgray'))
).properties(
  selection=interval
)

histogram = alt.Chart(cars).mark_bar().encode(
  x='count()',
  y='Origin',
  color='Origin'
).transform_filter(interval)

points & histogram

VConcatChart({
  vconcat: [Chart({
    data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
    0            12.0          8         307.0       130.0              18.0   
    1            11.5          8         350.0       165.0              15.0   
    2            11.0          8         318.0       150.0              18.0   
    3            12.0          8         304.0       150.0              16.0   
    4            10.5          8         302.0       140.0              17.0   
    5            10.0          8         429.0       198.0              15.0   
    6             9.0          8         454.0       220.0              14.0   
    7             8.5          8         440.0       215.0              14.0   
    8            10.0          8         455.0       225.0              14.0   
    9             8.5          8         390.0       190.0              15.0   
    10           17.5          4         133.0       115.0               NaN   
    11           11.5          8         350.0       165.0               NaN   
    12           11.0          8         351.0       153.0               NaN   
    13           10.5          8         383.0       175.0               NaN   
    14           11.0          8         360.0       175.0               NaN   
    15           10.0          8         383.0       170.0              15.0   
    16            8.0          8         340.0       160.0              14.0   
    17            8.0          8         302.0       140.0               NaN   
    18            9.5          8         400.0       150.0              15.0   
    19           10.0          8         455.0       225.0              14.0   
    20           15.0          4         113.0        95.0              24.0   
    21           15.5          6         198.0        95.0              22.0   
    22           15.5          6         199.0        97.0              18.0   
    23           16.0          6         200.0        85.0              21.0   
    24           14.5          4          97.0        88.0              27.0   
    25           20.5          4          97.0        46.0              26.0   
    26           17.5          4         110.0        87.0              25.0   
    27           14.5          4         107.0        90.0              24.0   
    28           17.5          4         104.0        95.0              25.0   
    29           12.5          4         121.0       113.0              26.0   
    ..            ...        ...           ...         ...               ...   
    376          18.6          4         112.0        88.0              27.0   
    377          18.0          4         112.0        88.0              34.0   
    378          16.2          4         112.0        85.0              31.0   
    379          16.0          4         135.0        84.0              29.0   
    380          18.0          4         151.0        90.0              27.0   
    381          16.4          4         140.0        92.0              24.0   
    382          20.5          4         151.0         NaN              23.0   
    383          15.3          4         105.0        74.0              36.0   
    384          18.2          4          91.0        68.0              37.0   
    385          17.6          4          91.0        68.0              31.0   
    386          14.7          4         105.0        63.0              38.0   
    387          17.3          4          98.0        70.0              36.0   
    388          14.5          4         120.0        88.0              36.0   
    389          14.5          4         107.0        75.0              36.0   
    390          16.9          4         108.0        70.0              34.0   
    391          15.0          4          91.0        67.0              38.0   
    392          15.7          4          91.0        67.0              32.0   
    393          16.2          4       

In [70]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

interval = alt.selection_interval()

base = alt.Chart(cars).mark_point().encode(
  y='Miles_per_Gallon',
  color=alt.condition(interval, 'Origin', alt.value('lightgray'))
).properties(
  selection=interval
)

base.encode(x='Acceleration') | base.encode(x='Horsepower')

HConcatChart({
  hconcat: [Chart({
    data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
    0            12.0          8         307.0       130.0              18.0   
    1            11.5          8         350.0       165.0              15.0   
    2            11.0          8         318.0       150.0              18.0   
    3            12.0          8         304.0       150.0              16.0   
    4            10.5          8         302.0       140.0              17.0   
    5            10.0          8         429.0       198.0              15.0   
    6             9.0          8         454.0       220.0              14.0   
    7             8.5          8         440.0       215.0              14.0   
    8            10.0          8         455.0       225.0              14.0   
    9             8.5          8         390.0       190.0              15.0   
    10           17.5          4         133.0       115.0               NaN   
    11           11.5          8         350.0       165.0               NaN   
    12           11.0          8         351.0       153.0               NaN   
    13           10.5          8         383.0       175.0               NaN   
    14           11.0          8         360.0       175.0               NaN   
    15           10.0          8         383.0       170.0              15.0   
    16            8.0          8         340.0       160.0              14.0   
    17            8.0          8         302.0       140.0               NaN   
    18            9.5          8         400.0       150.0              15.0   
    19           10.0          8         455.0       225.0              14.0   
    20           15.0          4         113.0        95.0              24.0   
    21           15.5          6         198.0        95.0              22.0   
    22           15.5          6         199.0        97.0              18.0   
    23           16.0          6         200.0        85.0              21.0   
    24           14.5          4          97.0        88.0              27.0   
    25           20.5          4          97.0        46.0              26.0   
    26           17.5          4         110.0        87.0              25.0   
    27           14.5          4         107.0        90.0              24.0   
    28           17.5          4         104.0        95.0              25.0   
    29           12.5          4         121.0       113.0              26.0   
    ..            ...        ...           ...         ...               ...   
    376          18.6          4         112.0        88.0              27.0   
    377          18.0          4         112.0        88.0              34.0   
    378          16.2          4         112.0        85.0              31.0   
    379          16.0          4         135.0        84.0              29.0   
    380          18.0          4         151.0        90.0              27.0   
    381          16.4          4         140.0        92.0              24.0   
    382          20.5          4         151.0         NaN              23.0   
    383          15.3          4         105.0        74.0              36.0   
    384          18.2          4          91.0        68.0              37.0   
    385          17.6          4          91.0        68.0              31.0   
    386          14.7          4         105.0        63.0              38.0   
    387          17.3          4          98.0        70.0              36.0   
    388          14.5          4         120.0        88.0              36.0   
    389          14.5          4         107.0        75.0              36.0   
    390          16.9          4         108.0        70.0              34.0   
    391          15.0          4          91.0        67.0              38.0   
    392          15.7          4          91.0        67.0              32.0   
    393          16.2          4       

In [71]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_point().encode(
  x='Horsepower',
  y='Miles_per_Gallon',
  color='Origin'
).interactive()

Chart({
  data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
  0            12.0          8         307.0       130.0              18.0   
  1            11.5          8         350.0       165.0              15.0   
  2            11.0          8         318.0       150.0              18.0   
  3            12.0          8         304.0       150.0              16.0   
  4            10.5          8         302.0       140.0              17.0   
  5            10.0          8         429.0       198.0              15.0   
  6             9.0          8         454.0       220.0              14.0   
  7             8.5          8         440.0       215.0              14.0   
  8            10.0          8         455.0       225.0              14.0   
  9             8.5          8         390.0       190.0              15.0   
  10           17.5          4         133.0       115.0               NaN   
  11           11.5          8         350.0       165.0               NaN   
  12           11.0          8         351.0       153.0               NaN   
  13           10.5          8         383.0       175.0               NaN   
  14           11.0          8         360.0       175.0               NaN   
  15           10.0          8         383.0       170.0              15.0   
  16            8.0          8         340.0       160.0              14.0   
  17            8.0          8         302.0       140.0               NaN   
  18            9.5          8         400.0       150.0              15.0   
  19           10.0          8         455.0       225.0              14.0   
  20           15.0          4         113.0        95.0              24.0   
  21           15.5          6         198.0        95.0              22.0   
  22           15.5          6         199.0        97.0              18.0   
  23           16.0          6         200.0        85.0              21.0   
  24           14.5          4          97.0        88.0              27.0   
  25           20.5          4          97.0        46.0              26.0   
  26           17.5          4         110.0        87.0              25.0   
  27           14.5          4         107.0        90.0              24.0   
  28           17.5          4         104.0        95.0              25.0   
  29           12.5          4         121.0       113.0              26.0   
  ..            ...        ...           ...         ...               ...   
  376          18.6          4         112.0        88.0              27.0   
  377          18.0          4         112.0        88.0              34.0   
  378          16.2          4         112.0        85.0              31.0   
  379          16.0          4         135.0        84.0              29.0   
  380          18.0          4         151.0        90.0              27.0   
  381          16.4          4         140.0        92.0              24.0   
  382          20.5          4         151.0         NaN              23.0   
  383          15.3          4         105.0        74.0              36.0   
  384          18.2          4          91.0        68.0              37.0   
  385          17.6          4          91.0        68.0              31.0   
  386          14.7          4         105.0        63.0              38.0   
  387          17.3          4          98.0        70.0              36.0   
  388          14.5          4         120.0        88.0              36.0   
  389          14.5          4         107.0        75.0              36.0   
  390          16.9          4         108.0        70.0              34.0   
  391          15.0          4          91.0        67.0              38.0   
  392          15.7          4          91.0        67.0              32.0   
  393          16.2          4          91.0        67.0              38.0   
  394          16.4          6         181.0       110.0              25.0   
  395   

In [72]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

interval = alt.selection_interval()

alt.Chart(cars).mark_point().encode(
  x='Horsepower',
  y='Miles_per_Gallon',
  color=alt.condition(interval, 'Origin', alt.value('lightgray'))
).properties(
  selection=interval
)

Chart({
  data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
  0            12.0          8         307.0       130.0              18.0   
  1            11.5          8         350.0       165.0              15.0   
  2            11.0          8         318.0       150.0              18.0   
  3            12.0          8         304.0       150.0              16.0   
  4            10.5          8         302.0       140.0              17.0   
  5            10.0          8         429.0       198.0              15.0   
  6             9.0          8         454.0       220.0              14.0   
  7             8.5          8         440.0       215.0              14.0   
  8            10.0          8         455.0       225.0              14.0   
  9             8.5          8         390.0       190.0              15.0   
  10           17.5          4         133.0       115.0               NaN   
  11           11.5          8         350.0       165.0               NaN   
  12           11.0          8         351.0       153.0               NaN   
  13           10.5          8         383.0       175.0               NaN   
  14           11.0          8         360.0       175.0               NaN   
  15           10.0          8         383.0       170.0              15.0   
  16            8.0          8         340.0       160.0              14.0   
  17            8.0          8         302.0       140.0               NaN   
  18            9.5          8         400.0       150.0              15.0   
  19           10.0          8         455.0       225.0              14.0   
  20           15.0          4         113.0        95.0              24.0   
  21           15.5          6         198.0        95.0              22.0   
  22           15.5          6         199.0        97.0              18.0   
  23           16.0          6         200.0        85.0              21.0   
  24           14.5          4          97.0        88.0              27.0   
  25           20.5          4          97.0        46.0              26.0   
  26           17.5          4         110.0        87.0              25.0   
  27           14.5          4         107.0        90.0              24.0   
  28           17.5          4         104.0        95.0              25.0   
  29           12.5          4         121.0       113.0              26.0   
  ..            ...        ...           ...         ...               ...   
  376          18.6          4         112.0        88.0              27.0   
  377          18.0          4         112.0        88.0              34.0   
  378          16.2          4         112.0        85.0              31.0   
  379          16.0          4         135.0        84.0              29.0   
  380          18.0          4         151.0        90.0              27.0   
  381          16.4          4         140.0        92.0              24.0   
  382          20.5          4         151.0         NaN              23.0   
  383          15.3          4         105.0        74.0              36.0   
  384          18.2          4          91.0        68.0              37.0   
  385          17.6          4          91.0        68.0              31.0   
  386          14.7          4         105.0        63.0              38.0   
  387          17.3          4          98.0        70.0              36.0   
  388          14.5          4         120.0        88.0              36.0   
  389          14.5          4         107.0        75.0              36.0   
  390          16.9          4         108.0        70.0              34.0   
  391          15.0          4          91.0        67.0              38.0   
  392          15.7          4          91.0        67.0              32.0   
  393          16.2          4          91.0        67.0              38.0   
  394          16.4          6         181.0       110.0              25.0   
  395   

In [73]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_bar().encode(
  x=alt.X('Miles_per_Gallon', bin=True),
  y='count()',
)

Chart({
  data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
  0            12.0          8         307.0       130.0              18.0   
  1            11.5          8         350.0       165.0              15.0   
  2            11.0          8         318.0       150.0              18.0   
  3            12.0          8         304.0       150.0              16.0   
  4            10.5          8         302.0       140.0              17.0   
  5            10.0          8         429.0       198.0              15.0   
  6             9.0          8         454.0       220.0              14.0   
  7             8.5          8         440.0       215.0              14.0   
  8            10.0          8         455.0       225.0              14.0   
  9             8.5          8         390.0       190.0              15.0   
  10           17.5          4         133.0       115.0               NaN   
  11           11.5          8         350.0       165.0               NaN   
  12           11.0          8         351.0       153.0               NaN   
  13           10.5          8         383.0       175.0               NaN   
  14           11.0          8         360.0       175.0               NaN   
  15           10.0          8         383.0       170.0              15.0   
  16            8.0          8         340.0       160.0              14.0   
  17            8.0          8         302.0       140.0               NaN   
  18            9.5          8         400.0       150.0              15.0   
  19           10.0          8         455.0       225.0              14.0   
  20           15.0          4         113.0        95.0              24.0   
  21           15.5          6         198.0        95.0              22.0   
  22           15.5          6         199.0        97.0              18.0   
  23           16.0          6         200.0        85.0              21.0   
  24           14.5          4          97.0        88.0              27.0   
  25           20.5          4          97.0        46.0              26.0   
  26           17.5          4         110.0        87.0              25.0   
  27           14.5          4         107.0        90.0              24.0   
  28           17.5          4         104.0        95.0              25.0   
  29           12.5          4         121.0       113.0              26.0   
  ..            ...        ...           ...         ...               ...   
  376          18.6          4         112.0        88.0              27.0   
  377          18.0          4         112.0        88.0              34.0   
  378          16.2          4         112.0        85.0              31.0   
  379          16.0          4         135.0        84.0              29.0   
  380          18.0          4         151.0        90.0              27.0   
  381          16.4          4         140.0        92.0              24.0   
  382          20.5          4         151.0         NaN              23.0   
  383          15.3          4         105.0        74.0              36.0   
  384          18.2          4          91.0        68.0              37.0   
  385          17.6          4          91.0        68.0              31.0   
  386          14.7          4         105.0        63.0              38.0   
  387          17.3          4          98.0        70.0              36.0   
  388          14.5          4         120.0        88.0              36.0   
  389          14.5          4         107.0        75.0              36.0   
  390          16.9          4         108.0        70.0              34.0   
  391          15.0          4          91.0        67.0              38.0   
  392          15.7          4          91.0        67.0              32.0   
  393          16.2          4          91.0        67.0              38.0   
  394          16.4          6         181.0       110.0              25.0   
  395   

In [74]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_bar().encode(
  x='mean(Miles_per_Gallon)',
  y='Origin',
  color='Origin'
)

Chart({
  data:      Acceleration  Cylinders  Displacement  Horsepower  Miles_per_Gallon  \
  0            12.0          8         307.0       130.0              18.0   
  1            11.5          8         350.0       165.0              15.0   
  2            11.0          8         318.0       150.0              18.0   
  3            12.0          8         304.0       150.0              16.0   
  4            10.5          8         302.0       140.0              17.0   
  5            10.0          8         429.0       198.0              15.0   
  6             9.0          8         454.0       220.0              14.0   
  7             8.5          8         440.0       215.0              14.0   
  8            10.0          8         455.0       225.0              14.0   
  9             8.5          8         390.0       190.0              15.0   
  10           17.5          4         133.0       115.0               NaN   
  11           11.5          8         350.0       165.0               NaN   
  12           11.0          8         351.0       153.0               NaN   
  13           10.5          8         383.0       175.0               NaN   
  14           11.0          8         360.0       175.0               NaN   
  15           10.0          8         383.0       170.0              15.0   
  16            8.0          8         340.0       160.0              14.0   
  17            8.0          8         302.0       140.0               NaN   
  18            9.5          8         400.0       150.0              15.0   
  19           10.0          8         455.0       225.0              14.0   
  20           15.0          4         113.0        95.0              24.0   
  21           15.5          6         198.0        95.0              22.0   
  22           15.5          6         199.0        97.0              18.0   
  23           16.0          6         200.0        85.0              21.0   
  24           14.5          4          97.0        88.0              27.0   
  25           20.5          4          97.0        46.0              26.0   
  26           17.5          4         110.0        87.0              25.0   
  27           14.5          4         107.0        90.0              24.0   
  28           17.5          4         104.0        95.0              25.0   
  29           12.5          4         121.0       113.0              26.0   
  ..            ...        ...           ...         ...               ...   
  376          18.6          4         112.0        88.0              27.0   
  377          18.0          4         112.0        88.0              34.0   
  378          16.2          4         112.0        85.0              31.0   
  379          16.0          4         135.0        84.0              29.0   
  380          18.0          4         151.0        90.0              27.0   
  381          16.4          4         140.0        92.0              24.0   
  382          20.5          4         151.0         NaN              23.0   
  383          15.3          4         105.0        74.0              36.0   
  384          18.2          4          91.0        68.0              37.0   
  385          17.6          4          91.0        68.0              31.0   
  386          14.7          4         105.0        63.0              38.0   
  387          17.3          4          98.0        70.0              36.0   
  388          14.5          4         120.0        88.0              36.0   
  389          14.5          4         107.0        75.0              36.0   
  390          16.9          4         108.0        70.0              34.0   
  391          15.0          4          91.0        67.0              38.0   
  392          15.7          4          91.0        67.0              32.0   
  393          16.2          4          91.0        67.0              38.0   
  394          16.4          6         181.0       110.0              25.0   
  395   

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
import pandas as pd

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[your Cloud Platform project ID]'
sample_count = 2000
row_count = pd.io.gbq.read_gbq('''
  SELECT 
    COUNT(*) as total
  FROM [bigquery-public-data:samples.gsod]''', project_id=project_id, verbose=False).total[0]
df = pd.io.gbq.read_gbq('''
  SELECT
    *
  FROM
    [bigquery-public-data:samples.gsod]
  WHERE RAND() < %d/%d
''' % (sample_count, row_count), project_id=project_id, verbose=False)
print('Full dataset has %d rows' % row_count)

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
import pandas as pd

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[your Cloud Platform project ID]'
sample_count = 2000
row_count = pd.io.gbq.read_gbq('''
  SELECT 
    COUNT(*) as total
  FROM [bigquery-public-data:samples.gsod]''', project_id=project_id, verbose=False).total[0]
df = pd.io.gbq.read_gbq('''
  SELECT
    *
  FROM
    [bigquery-public-data:samples.gsod]
  WHERE RAND() < %d/%d
''' % (sample_count, row_count), project_id=project_id, verbose=False)
print('Full dataset has %d rows' % row_count)

In [0]:
%%html
<link rel="stylesheet" href="/nbextensions/google.colab/tabbar.css">
<div class='goog-tab'>
  Some content
</div>

In [77]:
import portpicker
import threading
import socket
import IPython

from six.moves import socketserver
from six.moves import SimpleHTTPServer

class V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  def do_GET(self):
    self.send_response(200)
    # If the response should not be cached in the notebook for
    # offline access:
    # self.send_header('x-colab-notebook-cache-control', 'no-cache')
    self.end_headers()
    self.wfile.write(b'''
      document.querySelector('#output-area').appendChild(document.createTextNode('Script result!'));
    ''')

port = portpicker.pick_unused_port()

def server_entry():
    httpd = V6Server(('::', port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()

# Display some HTML referencing the resource.
display(IPython.display.HTML('<script src="https://localhost:{port}/"></script>'.format(port=port)))

In [78]:
import time
import sys
from google.colab import output

print('Starting.')

with output.use_tags('some_outputs'):
  sys.stdout.write('working....\n')
  sys.stdout.flush();
  time.sleep(2)

  sys.stdout.write('still working...\n')
  sys.stdout.flush();
  time.sleep(2)

# Now clear the previous outputs.
output.clear(output_tags='some_outputs')
print('All done!')


Starting.
All done!


In [79]:
# To determine which version you're using:
!pip show tensorflow

# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2

# For the latest nightly build:
!pip install tf-nightly

Name: tensorflow
Version: 1.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: protobuf, keras-preprocessing, numpy, astor, absl-py, wheel, gast, keras-applications, grpcio, six, termcolor, tensorboard
Required-by: stable-baselines, magenta, fancyimpute
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.12.0)
    100% |████████████████████████████████| 35.0MB 1.1MB/s 
    100% |████████████████████████████████| 890kB 16.1MB/s 
    100% |████████████████████████████████| 245kB 25.6MB/s 
  Running setup.py bdist_wheel for html5lib ... - \ done
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Running setup.py bdist_wheel for markdown ... - \ | / - done
  Stored in directory: /

In [81]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Flower_folder_rename_code.txt to Flower_folder_rename_code.txt
User uploaded file "Flower_folder_rename_code.txt" with length 2519 bytes


In [0]:
from google.colab import auth
auth.authenticate_user()

##########################
##########################

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

In [0]:
model = models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:02<00:00, 85715914.52it/s]


In [0]:
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

In [0]:
for param in model.parameters():
    param.requires_grad = False

In [0]:
model.classifier = classifier

In [0]:
def train_model(model, criteria, optimizer, scheduler,    
                                      num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
  
  


In [0]:
!wget -O flower_data_orginal_test.zip "https://www.dropbox.com/s/da6ye9genbsdzbq/flower_data_original_test.zip?dl=1"
!unzip flower_data_orginal_test.zip`

--2018-12-26 18:29:10--  https://www.dropbox.com/s/da6ye9genbsdzbq/flower_data_original_test.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/da6ye9genbsdzbq/flower_data_original_test.zip [following]
--2018-12-26 18:29:10--  https://www.dropbox.com/s/dl/da6ye9genbsdzbq/flower_data_original_test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0ae9ae36f63d1b5fe715a7131d.dl.dropboxusercontent.com/cd/0/get/AYMr1TmlSHQgeFCwWSNk0-3k6q-e43v4efG1QlsyeQEfqxraFAgdVO3dBHP1_dHNrb_M1jDmIFgnLs3AcNMBidi7SMJwuSn5CA7_N0WugXK3WVP_sqZvGKqPTZ274tlO2DJPMvo7wgoGSGyJxAJsK7pYolYdcUSHXQr4MJc3ioRHkVO8ACEb7K1Ej2CRoSDqvJY/file?dl=1 [following]
--2018-12-26 18:29:10--  https://uc0ae9ae36f63d1b5fe715a7131d.dl.dropboxusercontent.com/cd/0

In [0]:
!wget  https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json

--2018-12-26 18:31:25--  https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘cat_to_name.json’

cat_to_name.json    100%[===================>]   2.17K  --.-KB/s    in 0s      

2018-12-26 18:31:25 (37.1 MB/s) - ‘cat_to_name.json’ saved [2218/2218]



In [0]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in '%s': %s" % (cwd, files))


Files in '/content': ['.config', 'cat_to_name.json', 'flower_data_orginal_test.zip', 'sample_data']


In [0]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(255),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
!wget data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

--2018-12-26 19:17:14--  http://data_dir/
Resolving data_dir (data_dir)... failed: Name or service not known.
wget: unable to resolve host address ‘data_dir’
--2018-12-26 19:17:14--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2018-12-26 19:17:14--  http://flower_data/
Resolving flower_data (flower_data)... failed: Name or service not known.
wget: unable to resolve host address ‘flower_data’


In [0]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(valid_dir, transform=test_transforms)

'''Dataset split'''
batch_size = 64
valid_size = 0.1

FileNotFoundError: ignored

In [0]:
with open('cat_to_name.json', 'r') as f:
    label_map = json.load(f)

In [0]:
criteria = nn.NLLLoss()

In [0]:
import chainer.optimizers as O
optimizer = O.Adam()
